## <center> LGMVIP_DS_ September_23_Task_Number_3-2 <center>
## <center> Next Word Prediction <center>
## <center> By K. Pavan Kumar <center>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing Libraries

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk.tokenize import word_tokenize
import re
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Dropout, Activation, BatchNormalization, SimpleRNN
from tensorflow.keras.optimizers import RMSprop

In [3]:
file='/kaggle/input/next-word-prediction-dataset/1661-0.txt'

In [4]:
# reading file and conveting it to lower case
text=open(file).read().lower()

In [5]:
print('Length of Corpus is: ',len(text))

Length of Corpus is:  581888


In [6]:
words1=re.findall(r'[a-zA-Z]+',text)

In [7]:
# Extracting words only made up of alphabets
filtered_text=" ".join(re.findall(r'[a-zA-Z]+',text))

In [8]:
# Tokenizing filtered_text into words
words=word_tokenize(filtered_text)

In [9]:
unique_words=np.unique(words)

In [10]:
len(np.unique(words))

8081

In [11]:
# forming a dictionary which consist of unique words as index and index numbers as values
unique_words_dict=dict((j,i) for i, j in enumerate(unique_words))

## Feature Engineering:

In [12]:
# number_words signifies the sequence length of the sentence
number_words=5
prev_words=[] #initialising empty list of previous words
next_word=[] #initialising empty list of next words
for i in range(len(words)-number_words): #Ensures we only get sentences containing only 5 words
    prev_words.append(words[i:i+number_words])
    next_word.append(words[i+number_words])

In [13]:
print(prev_words[4])
print(next_word[4])

['adventures', 'of', 'sherlock', 'holmes', 'by']
arthur


In [14]:
#Initialising X and Y
#X with shape of 1st value suggesting number of samples, 2nd value suggesting number of words in sentence,
#3rd value suggesting length of unique words and we are converting it to boolean type
X=np.zeros((len(prev_words),number_words,len(unique_words)),dtype='bool')
Y=np.zeros((len(next_word),len(unique_words)),dtype='bool')

In [15]:
X.shape

(109071, 5, 8081)

In [16]:
Y.shape

(109071, 8081)

In [17]:
for i, words in enumerate(prev_words):
    for j , every_word in enumerate(words):
        # 'i' means we entered a sentence out of so many samples
        # 'j' means we entered a word in that sentence
        # Now we are forming a bag of words matrix for that word by assigning index of that word
        # from unique_words_dict a value of 1.
        X[i,j,unique_words_dict[every_word]]=1
    Y[i,unique_words_dict[next_word[i]]]=1

In [18]:
X[1].shape

(5, 8081)

In [19]:
np.where(X[1]==True)

(array([0, 1, 2, 3, 4]), array([3273, 6039, 7139,  132, 4844]))

## Model Building:

In [20]:
model=Sequential()

In [25]:
model.add(SimpleRNN(128,input_shape=(number_words,len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               1050880   
                                                                 
 dense (Dense)               (None, 8081)              1042449   
                                                                 
 activation (Activation)     (None, 8081)              0         
                                                                 
Total params: 2,093,329
Trainable params: 2,093,329
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

## Model Training:

In [28]:
history=model.fit(X,Y,validation_split=0.30,batch_size=128,epochs=30,shuffle=True)

Epoch 1/30
597/597 [==============================] - 19s 20ms/step - loss: 6.6755 - accuracy: 0.0546 - val_loss: 6.6801 - val_accuracy: 0.0560
Epoch 2/30
597/597 [==============================] - 7s 12ms/step - loss: 6.2118 - accuracy: 0.0688 - val_loss: 6.4966 - val_accuracy: 0.0668
Epoch 3/30
597/597 [==============================] - 8s 14ms/step - loss: 5.8382 - accuracy: 0.0979 - val_loss: 6.3534 - val_accuracy: 0.0910
Epoch 4/30
597/597 [==============================] - 8s 14ms/step - loss: 5.4753 - accuracy: 0.1242 - val_loss: 6.3329 - val_accuracy: 0.0967
Epoch 5/30
597/597 [==============================] - 8s 13ms/step - loss: 5.1439 - accuracy: 0.1490 - val_loss: 6.4191 - val_accuracy: 0.0964
Epoch 6/30
597/597 [==============================] - 7s 12ms/step - loss: 4.8233 - accuracy: 0.1759 - val_loss: 6.4501 - val_accuracy: 0.0939
Epoch 7/30
597/597 [==============================] - 8s 14ms/step - loss: 4.5097 - accuracy: 0.2063 - val_loss: 6.5650 - val_accuracy: 0.090

## Predicting Next Word:

In [29]:
def predict_next_word(input,best_word=5):
    input=input.lower()
    X=np.zeros((1,number_words,len(unique_words)))
    for i, word in enumerate(input.split()):
        X[0,i,unique_words_dict[word]]=1
    prediction=model.predict(X)
    return np.argpartition(prediction,-best_word)

In [34]:
possible_words=predict_next_word('author of sherlock holmes')[0][:5]

1/1 [==============================] - 0s 20ms/step


In [35]:
# The best possible 5 words predicted as next word by the model
[unique_words[i] for i in possible_words]

['murmured', 'zip', 'zigzag', 'abbots', 'aberdeen']